In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import json
import pickle

import scipy
from scipy import constants
from scipy import stats
import astropy.constants


from wtf import wtf_compile

2025-09-06 16:17:14.624085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-06 16:17:14.635925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-06 16:17:14.639430: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-06 16:17:15.260278: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
with open('pitchfork/pitchfork.json', 'r') as fp:
    pitchfork_dict = json.load(fp)

pitchfork = wtf_compile(pitchfork_dict, jaxxed=False)

jpitchfork = wtf_compile(pitchfork_dict, jaxxed=True)

with open('pitchfork/pitchfork_info.json', 'r') as fp:
    pitchfork_info = json.load(fp)

pitchfork_cov = np.loadtxt('pitchfork/pitchfork_covariance.txt')

In [14]:
pca_comps = np.array(pitchfork_info['custom_objects']['inverse_pca']['pca_comps'])
pca_mean = np.array(pitchfork_info['custom_objects']['inverse_pca']['pca_mean'])

In [15]:
preds = pitchfork.forward(np.array([[0.5,0.5,0.5,0.5,0.5]]))
preds

[array([[0.45746291, 0.55022404, 0.37583447]]),
 array([[ 2.42869059e+00, -2.09924402e-02, -2.91607387e-02,
         -6.30696148e-03, -1.37554752e-03,  1.22286922e-03,
          4.84569803e-03,  2.07647610e-03,  2.68109677e-03,
          9.62887053e-04, -6.12263785e-04,  1.05727241e-03,
          1.32305702e-03, -1.92868063e-03, -1.74395222e-03]])]

In [22]:
def pcann(x, jaxxed=False):
    if jaxxed:
        y = jnp.tensordot(x, pca_comps, 1) + pca_mean
        return y
    else:
        y = np.tensordot(x, pca_comps, 1) + pca_mean
        return y


def predict(x, emulator, jaxxed=False):
    if jaxxed:
        log_inputs_mean = jnp.array(pitchfork_info["data_scaling"]["inp_mean"][0])
        
        log_inputs_std = jnp.array(pitchfork_info["data_scaling"]["inp_std"][0])

        log_outputs_mean = jnp.array(pitchfork_info["data_scaling"]["classical_out_mean"][0] + pitchfork_info["data_scaling"]["astero_out_mean"][0])
        
        log_outputs_std = jnp.array(pitchfork_info["data_scaling"]["classical_out_std"][0] + pitchfork_info["data_scaling"]["astero_out_std"][0])
            
        log_inputs = jnp.log10(x)
        
        standardised_log_inputs = (log_inputs - log_inputs_mean)/log_inputs_std

        preds = jpitchfork.forward(standardised_log_inputs)
        
        standardised_log_outputs = jnp.concatenate((jnp.array(preds[0]),jnp.array(pcann(preds[1], jaxxed=jaxxed))), axis=1)

        log_outputs = (standardised_log_outputs*log_outputs_std) + log_outputs_mean

        outputs = 10**log_outputs

        outputs = outputs.at[:,2].set(log_outputs[:,2]) ##we want star_feh in dex

        teff = jnp.array(((outputs[:,1]*astropy.constants.L_sun) / (4*jnp.pi*constants.sigma*((outputs[:,0]*astropy.constants.R_sun)**2)))**0.25)
        
        outputs = outputs.at[:,0].set(teff)
        
        #outputs = np.concatenate((np.array(outputs[:,:3]), np.array(outputs[:,n_min-3:n_max-2])), axis=1)

        return outputs
        
    else:
        log_inputs_mean = np.array(pitchfork_info["data_scaling"]["inp_mean"][0])
        
        log_inputs_std = np.array(pitchfork_info["data_scaling"]["inp_std"][0])

        log_outputs_mean = np.array(pitchfork_info["data_scaling"]["classical_out_mean"][0] + pitchfork_info["data_scaling"]["astero_out_mean"][0])
        
        log_outputs_std = np.array(pitchfork_info["data_scaling"]["classical_out_std"][0] + pitchfork_info["data_scaling"]["astero_out_std"][0])
            
        log_inputs = np.log10(x)
        
        standardised_log_inputs = (log_inputs - log_inputs_mean)/log_inputs_std

        preds = pitchfork.forward(standardised_log_inputs)
        
        standardised_log_outputs = np.concatenate((np.array(preds[0]),np.array(pcann(preds[1], jaxxed=jaxxed))), axis=1)

        log_outputs = (standardised_log_outputs*log_outputs_std) + log_outputs_mean

        outputs = 10**log_outputs

        outputs[:,2] = log_outputs[:,2] ##we want star_feh in dex

        teff = np.array(((outputs[:,1]*astropy.constants.L_sun) / (4*np.pi*constants.sigma*((outputs[:,0]*astropy.constants.R_sun)**2)))**0.25)
        
        outputs[:,0] = teff
        
        #outputs = np.concatenate((np.array(outputs[:,:3]), np.array(outputs[:,n_min-3:n_max-2])), axis=1)

        return outputs
        

In [17]:
predict(np.array([[0.5,0.5,0.5,0.5,0.5]]), pitchfork)

array([[2.90103052e+03, 6.20954018e-02, 1.68595571e+00, 7.08014276e+03,
        3.19778967e+03, 1.91677524e+03, 3.10554663e+03, 4.67799255e+03,
        3.58019538e+03, 2.92512615e+03, 4.12115449e+03, 6.41977217e+03,
        7.11880497e+03, 6.16948384e+03, 5.66950715e+03, 6.24773686e+03,
        7.53749071e+03, 8.78368790e+03, 9.43642634e+03, 9.44136905e+03,
        9.04100965e+03, 8.58677296e+03, 8.17628446e+03, 7.67193348e+03,
        7.00492316e+03, 6.56804817e+03, 6.88645134e+03, 8.24118060e+03,
        1.04124600e+04, 1.23062569e+04, 1.27352167e+04, 1.18730838e+04,
        1.08532563e+04, 1.04488193e+04, 1.07413303e+04, 1.14497843e+04,
        1.22779177e+04, 1.30732578e+04]])

## checking same as tf_model predictions

In [7]:
single_point = np.loadtxt('single_point.txt')
million_points = np.loadtxt('million_points.txt')

tf_single_pred = np.loadtxt('single_pred.txt')
tf_million_preds = np.loadtxt('million_preds.txt')

In [18]:
%%time
wtf_single_pred = predict((single_point,), pitchfork)

CPU times: user 12.1 ms, sys: 3 μs, total: 12.1 ms
Wall time: 11.8 ms


In [19]:
%%time
wtf_million_preds = predict(million_points, pitchfork)

CPU times: user 4min 21s, sys: 3.66 s, total: 4min 25s
Wall time: 14.9 s


In [72]:
for i in range(38):
    residuals = wtf_million_preds[:,i] - tf_million_preds[:,i]
    print(f'max is {residuals.mean()} for output_idx: {i}')

max is 0.3264801458105787 for output_idx: 0
max is 0.0003017432976775625 for output_idx: 1
max is 5.280508935306782e-05 for output_idx: 2
max is 0.025187219148772977 for output_idx: 3
max is 0.02979354004162522 for output_idx: 4
max is 0.027431318265633355 for output_idx: 5
max is 0.03289065259672762 for output_idx: 6
max is 0.040430318707963694 for output_idx: 7
max is 0.034914506904937845 for output_idx: 8
max is 0.022977868716477168 for output_idx: 9
max is 0.022430189777699536 for output_idx: 10
max is 0.03258313786677054 for output_idx: 11
max is 0.03966096681930552 for output_idx: 12
max is 0.03881206760777698 for output_idx: 13
max is 0.03676458832287041 for output_idx: 14
max is 0.03901874472433785 for output_idx: 15
max is 0.043843065300135625 for output_idx: 16
max is 0.04726047655268616 for output_idx: 17
max is 0.04756478828607069 for output_idx: 18
max is 0.04584842672819137 for output_idx: 19
max is 0.04556065424802895 for output_idx: 20
max is 0.04730030490732655 for out

## check jwtf etc

In [23]:
jpredict = jax.jit(predict, static_argnums=1, static_argnames='jaxxed')

In [55]:
j_single_point = jnp.array((np.loadtxt('single_point.txt'),))
j_million_points = jnp.array(np.loadtxt('million_points.txt'))

j_tf_single_pred = jnp.array(np.loadtxt('single_pred.txt'))
j_tf_million_preds = jnp.array(np.loadtxt('million_preds.txt'))

In [31]:
%%time
j_single_pred = jpredict(j_single_point, jpitchfork, jaxxed=True)

CPU times: user 1.16 ms, sys: 4 μs, total: 1.16 ms
Wall time: 787 μs


In [56]:
%%time
j_million_preds = jpredict(j_million_points, jpitchfork, jaxxed=True)

CPU times: user 24.9 ms, sys: 27.1 ms, total: 52.1 ms
Wall time: 9.07 ms


In [61]:
j_million_preds[:,1]

Array(5.823207, dtype=float32)

In [71]:
for i in range(38):
    residuals = (j_million_preds[:,i] - j_tf_million_preds[:,i])
    print(f'max is {residuals.mean()} for output_idx: {i}')

max is 0.32660895586013794 for output_idx: 0
max is 0.00030190302641130984 for output_idx: 1
max is 5.2873358072247356e-05 for output_idx: 2
max is 0.025310326367616653 for output_idx: 3
max is 0.02949388697743416 for output_idx: 4
max is 0.027463145554065704 for output_idx: 5
max is 0.032495710998773575 for output_idx: 6
max is 0.040026579052209854 for output_idx: 7
max is 0.03511136397719383 for output_idx: 8
max is 0.022803984582424164 for output_idx: 9
max is 0.022184880450367928 for output_idx: 10
max is 0.032263822853565216 for output_idx: 11
max is 0.03954284265637398 for output_idx: 12
max is 0.03856853023171425 for output_idx: 13
max is 0.03684990108013153 for output_idx: 14
max is 0.039217039942741394 for output_idx: 15
max is 0.04359074681997299 for output_idx: 16
max is 0.04720724746584892 for output_idx: 17
max is 0.04702378064393997 for output_idx: 18
max is 0.04505884274840355 for output_idx: 19
max is 0.046020761132240295 for output_idx: 20
max is 0.046667445451021194 f

In [69]:
tf_single_pred.dtype

dtype('float64')

In [73]:
for i in range(38):
    residuals = (j_million_preds[:,i] - wtf_million_preds[:,i])
    print(f'max is {residuals.mean()} for output_idx: {i}')

max is 0.00012880712165497243 for output_idx: 0
max is 1.597288701304933e-07 for output_idx: 1
max is 6.826086007549748e-08 for output_idx: 2
max is 0.00012305844575166702 for output_idx: 3
max is -0.0002996966941282153 for output_idx: 4
max is 3.1778228731127456e-05 for output_idx: 5
max is -0.0003949769597966224 for output_idx: 6
max is -0.00040382082806900144 for output_idx: 7
max is 0.00019693374633789062 for output_idx: 8
max is -0.000173897497006692 for output_idx: 9
max is -0.0002453056222293526 for output_idx: 10
max is -0.00031939090695232153 for output_idx: 11
max is -0.00011807891860371456 for output_idx: 12
max is -0.0002435341157251969 for output_idx: 13
max is 8.537109533790499e-05 for output_idx: 14
max is 0.00019823663751594722 for output_idx: 15
max is -0.000252315541729331 for output_idx: 16
max is -5.309674088493921e-05 for output_idx: 17
max is -0.0005411254824139178 for output_idx: 18
max is -0.0007894913433119655 for output_idx: 19
max is 0.0004598613304551691 for